<a href="https://colab.research.google.com/github/Kabdiyev/YOLOv12/blob/main/Yolov12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import glob
from ultralytics import YOLO
import os
import gc
import cv2
import sqlite3
import numpy as np
from datetime import datetime
import easyocr

In [2]:
yaml_path = glob.glob('dataset/**/data.yaml', recursive=True)[0]
yaml_path

'dataset\\data.yaml'

In [3]:
import torch
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Device:", torch.cuda.get_device_name(0))
    print("CUDA version (runtime):", torch.version.cuda)
print("Torch:", torch.__version__)

CUDA available: True
Device: NVIDIA GeForce RTX 3060
CUDA version (runtime): 11.8
Torch: 2.7.1+cu118


In [ ]:


# 4) Обучение YOLOv12 (минимальный пример) + per-epoch тест для отслеживания overfitting
# Если в твоей версии ultralytics нет весов yolo12n.pt, замени на "yolo11n.pt".
model = YOLO("yolo12n.pt")

# История mAP50 по тесту на каждой эпохе
# Глобальный список, чтобы использовать позже в графике
try:
    test_map50_history
except NameError:
    test_map50_history = []

# Регистрируем коллбэк для валидации на тесте после каждой эпохи (отключено из-за конфликта автограда)
if False and not hasattr(model, "_test_cb_added"):
    def _on_fit_epoch_end(trainer):
        # Переключаем только тренировочную модель тренера в eval на время теста
        torch.cuda.synchronize() if torch.cuda.is_available() else None
        was_training = True
        try:
            if hasattr(trainer, "model") and hasattr(trainer.model, "training"):
                was_training = trainer.model.training
            if hasattr(trainer, "model") and hasattr(trainer.model, "eval"):
                trainer.model.eval()
            # Оцениваем на тестовом сплите из data.yaml под no_grad
            with torch.no_grad():
                metrics = model.val(
                    data=yaml_path,
                    split="test",
                    iou=0.5,
                    conf=0.001,
                    device=0,
                    plots=False,
                    verbose=False,
                )
        finally:
            # Возвращаем состояние тренировки
            if hasattr(trainer, "model") and hasattr(trainer.model, "train") and was_training:
                trainer.model.train()
        
        # Пытаемся извлечь mAP50
        map50 = None
        try:
            map50 = float(getattr(metrics.box, "map50"))
        except Exception:
            try:
                rd = getattr(metrics, "results_dict", {}) or {}
                for k in ("metrics/box/mAP50", "metrics/mAP50(B)", "metrics/mAP50"):
                    if k in rd:
                        map50 = float(rd[k])
                        break
            except Exception:
                map50 = None
        if map50 is not None:
            test_map50_history.append(map50)
    
    model.add_callback("on_fit_epoch_end", _on_fit_epoch_end)
    model._test_cb_added = True

# Запуск обучения
# Безопасность: включаем автоград и режим тренировки, чистим наш тест-коллбэк
try:
    torch.set_grad_enabled(True)
    if hasattr(model, 'model'):
        model.model.train()
    if hasattr(model, 'callbacks') and 'on_fit_epoch_end' in model.callbacks:
        model.callbacks['on_fit_epoch_end'] = [cb for cb in model.callbacks['on_fit_epoch_end'] if getattr(cb, '__name__', '') != '_on_fit_epoch_end']
except Exception:
    pass

results = model.train(
    data=yaml_path,   # путь к data.yaml из шага 3
    epochs=200,       # можно увеличить
    imgsz=640,        # 640 обычно достаточно
    device=0,         # GPU
    batch=8,          # снизим batch для RAM
    workers=2,        # меньше потоков загрузки
    cache=False,      # не кэшировать датасет в RAM
    val=True          # используем встроенную валидацию по эпохам
)



Ultralytics 8.3.203  Python-3.13.1 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset\data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo12n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train9, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=1

In [3]:
model = YOLO(r"runs\detect\train9\weights\best.pt")

In [ ]:
# 5) Инференс на видео (стримингово, без накопления в RAM)

# Очистка возможных старых результатов из памяти
try:
    pred = None
except NameError:
    pass
import gc; gc.collect()

# Потоковая обработка: результаты не накапливаются в переменной
for _ in model(
    source="IMG_6174.MP4",  # твоё видео
    stream=True,
    conf=0.8,               # порог детекции
    iou=0.5,
    device=0,
    save=True               # сохранит результат в runs/detect/predict*/IMG_6174.mp4
):
    pass


WARNING 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/2176) d:\Documents\YOLOv12\IMG_6174.MP4: 640x640 (no detections), 21.6ms
video 1/1 (frame 2/2176) d:\Documents\YOLOv12\IMG_6174.MP4: 640x640 (no detections), 22.1ms
video 1/1 (frame 3/2176) d:\Documents\YOLOv12\IMG_6174.MP4: 640x640 (no detections), 24.8ms
video 1/1 (frame 4/2176) d:\Documents\YOLOv12\IMG_6174.MP4: 640x640 (no detections), 22.4ms
video 1/1 (frame 5/2176) d:\Documents\YOLOv12\IMG_6174.MP4: 640x640 (no detections), 22.9ms

In [9]:
# 5.1) Сохранение кадров и YOLO-лейблов для детекций c conf > 0.8
# Папки назначения: dataset/inference/images и dataset/inference/labels

from pathlib import Path
import os
import cv2
import numpy as np

# Настройки
save_root = Path("dataset/inference")
images_dir = save_root / "images"
labels_dir = save_root / "labels"
images_dir.mkdir(parents=True, exist_ok=True)
labels_dir.mkdir(parents=True, exist_ok=True)

# Укажи путь к тому же видео, что и в шаге 5
video_path = "IMG_6174.MP4"  # замените при необходимости на ваш путь

# Порог уверенности и прочие параметры должны соответствовать использованным в шаге 5
conf_thresh = 0.8

# Потоковый инференс — не накапливает все результаты в памяти
results_gen = model(source=video_path, stream=True, conf=conf_thresh, iou=0.5, device=0)

frame_index = 0
for r in results_gen:
    # Исходный кадр BGR
    frame = r.orig_img

    boxes = r.boxes  # Boxes object
    has_boxes = boxes is not None and len(boxes) > 0
    if not has_boxes:
        frame_index += 1
        continue

    # Фильтруем по conf > 0.8
    confs = boxes.conf.cpu().numpy().reshape(-1)
    keep_mask = confs > conf_thresh
    if not np.any(keep_mask):
        frame_index += 1
        continue

    # Нормализованные боксы (cx, cy, w, h) в долях от размеров кадра
    xywhn = boxes.xywhn.cpu().numpy()[keep_mask]
    clss = boxes.cls.cpu().numpy().astype(int)[keep_mask]

    # Имя файла кадра и соответствующего лейбла
    img_name = f"frame_{frame_index:06d}.jpg"
    img_path = images_dir / img_name
    lbl_path = labels_dir / f"frame_{frame_index:06d}.txt"

    # Необязательное уменьшение размера кадра перед сохранением (сэкономит RAM/диск)
    max_w = 1280
    if frame.shape[1] > max_w:
        scale = max_w / frame.shape[1]
        frame = cv2.resize(frame, (max_w, int(frame.shape[0] * scale)), interpolation=cv2.INTER_AREA)

    # Сохраняем изображение
    cv2.imwrite(str(img_path), frame)

    # Сохраняем YOLO-формат лейблов: <class> <cx> <cy> <w> <h> (все нормализовано)
    with open(lbl_path, "w", encoding="utf-8") as f:
        for cls_id, (cx, cy, w, h) in zip(clss, xywhn):
            f.write(f"{int(cls_id)} {cx:.6f} {cy:.6f} {w:.6f} {h:.6f}\n")

    frame_index += 1

print(f"Готово. Кадры сохранены в: {images_dir}")
print(f"          Лейблы сохранены в: {labels_dir}")



video 1/1 (frame 1/2176) d:\Documents\YOLOv12\IMG_6174.MP4: 640x640 (no detections), 75.2ms
video 1/1 (frame 2/2176) d:\Documents\YOLOv12\IMG_6174.MP4: 640x640 (no detections), 23.1ms
video 1/1 (frame 3/2176) d:\Documents\YOLOv12\IMG_6174.MP4: 640x640 (no detections), 22.9ms
video 1/1 (frame 4/2176) d:\Documents\YOLOv12\IMG_6174.MP4: 640x640 (no detections), 24.8ms
video 1/1 (frame 5/2176) d:\Documents\YOLOv12\IMG_6174.MP4: 640x640 (no detections), 22.2ms
video 1/1 (frame 6/2176) d:\Documents\YOLOv12\IMG_6174.MP4: 640x640 (no detections), 23.6ms
video 1/1 (frame 7/2176) d:\Documents\YOLOv12\IMG_6174.MP4: 640x640 (no detections), 25.4ms
video 1/1 (frame 8/2176) d:\Documents\YOLOv12\IMG_6174.MP4: 640x640 (no detections), 23.2ms
video 1/1 (frame 9/2176) d:\Documents\YOLOv12\IMG_6174.MP4: 640x640 (no detections), 30.7ms
video 1/1 (frame 10/2176) d:\Documents\YOLOv12\IMG_6174.MP4: 640x640 (no detections), 23.4ms
video 1/1 (frame 11/2176) d:\Documents\YOLOv12\IMG_6174.MP4: 640x640 (no detec

In [11]:
# 4.1) График mAP50 по эпохам (тестовый сплит)
import matplotlib.pyplot as plt

if not test_map50_history:
    print("test_map50_history пуст. Сначала запусти обучение.")
else:
    plt.figure(figsize=(8,4))
    plt.plot(range(1, len(test_map50_history)+1), test_map50_history, marker='o', label='Test mAP50')
    plt.xlabel('Epoch')
    plt.ylabel('mAP50')
    plt.title('Test mAP50 per epoch')
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.show()


<Figure size 800x400 with 1 Axes>

In [11]:
# 5.2) Распознавание 8-значных номеров вагонов и сохранение в базу (SQLite)
# Потоковый инференс + OCR (EasyOCR). Уникальные 8-значные номера сохраняются в БД.

# Если EasyOCR не установлен:
# PowerShell:
# .\venv\Scripts\python.exe -m pip install easyocr

# Параметры
video_path_ocr = "IMG_6174.MP4"   # укажите путь к видео, как в шаге 5
conf_thresh_ocr = 0.85              # фильтр детекций для OCR
save_crops = True                  # сохранять вырезы с номерами
crops_dir = os.path.join("dataset", "inference", "crops")
os.makedirs(crops_dir, exist_ok=True)

# OCR инициализация: PaddleOCR (глубокая модель) или EasyOCR как фоллбэк
paddle_ocr = None
try:
    from paddleocr import PaddleOCR
    paddle_ocr = PaddleOCR(det=True, rec=True, use_angle_cls=True, lang='en', show_log=False)
except Exception:
    paddle_ocr = None

reader = None
try:
    import easyocr
    reader = easyocr.Reader(['en'], gpu=torch.cuda.is_available())
except Exception:
    reader = None

def ocr_digits_from_crop(crop_bgr):
    # Сначала пытаемся PaddleOCR
    if paddle_ocr is not None:
        try:
            crop_rgb = cv2.cvtColor(crop_bgr, cv2.COLOR_BGR2RGB)
            res = paddle_ocr.ocr(crop_rgb, cls=True)
            texts = []
            for line in (res or []):
                for det in (line or []):
                    txt, score = det[1][0], det[1][1]
                    if score is None or score >= 0.3:
                        texts.append(txt)
            if texts:
                return "".join(texts)
        except Exception:
            pass
    # Фоллбэк EasyOCR
    if reader is not None:
        try:
            return "".join(reader.readtext(crop_bgr, detail=0, paragraph=False, allowlist='0123456789'))
        except Exception:
            pass
    return ""

# Инициализация БД
db_path = os.path.join("dataset", "inference", "train_numbers.db")
conn = sqlite3.connect(db_path)
cur = conn.cursor()
cur.execute(
    """
    CREATE TABLE IF NOT EXISTS train_numbers (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        number TEXT UNIQUE,
        captured_at TEXT,
        video_path TEXT,
        frame_index INTEGER,
        x_center REAL,
        y_center REAL,
        width REAL,
        height REAL,
        conf REAL,
        crop_path TEXT
    );
    """
)
conn.commit()

# Вспомогательные функции
DIGITS = set("0123456789")

def only_digits(text: str) -> str:
    return "".join(ch for ch in text if ch in DIGITS)

# Основной цикл: потоковый инференс + OCR по ROI
frame_index = 0
numbers_seen = set()  # для ускорения – не обрабатывать один и тот же номер в текущем запуске

for r in model(source=video_path_ocr, stream=True, conf=conf_thresh_ocr, iou=0.5, device=0):
    frame = r.orig_img  # BGR
    h, w = frame.shape[:2]

    boxes = getattr(r, 'boxes', None)
    if boxes is None or len(boxes) == 0:
        frame_index += 1
        continue

    xyxy = boxes.xyxy.cpu().numpy()
    confs = boxes.conf.cpu().numpy().reshape(-1)
    xywhn = boxes.xywhn.cpu().numpy()

    for i, (x1, y1, x2, y2) in enumerate(xyxy):
        conf = float(confs[i])
        if conf < conf_thresh_ocr:
            continue

        # Расширяем ROI по ширине в 1.5 раза и клипуем в пределах кадра
        x_center = (x1 + x2) / 2.0
        width = (x2 - x1) * 1.5
        nx1 = int(max(0, x_center - width / 2.0))
        nx2 = int(min(w, x_center + width / 2.0))
        ny1 = max(0, int(y1))
        ny2 = min(h, int(y2))
        if nx2 <= nx1 or ny2 <= ny1:
            continue

        crop = frame[ny1:ny2, nx1:nx2]
        if crop.size == 0:
            continue
        # Делаем копию для гарантий непрерывности памяти (некоторые версии OpenCV это требуют)
        crop = crop.copy()

        # OCR с приоритетом PaddleOCR, фоллбэк EasyOCR
        text_all = ocr_digits_from_crop(crop)
        digits = only_digits(text_all)
        if len(digits) != 8:
            continue

        # Извлекаем нормализованные координаты для записи в БД
        cx, cy, ww, hh = map(float, xywhn[i])

        # Пропускаем, если уже видели в этом запуске
        if digits in numbers_seen:
            continue

        numbers_seen.add(digits)

        # Сохраняем кроп по желанию (и только если запись прошла успешно)
        crop_path = None
        if save_crops:
            try:
                os.makedirs(crops_dir, exist_ok=True)
                crop_name = f"frame_{frame_index:06d}_i{i}_{digits}.jpg"
                candidate_path = os.path.join(crops_dir, crop_name)
                ok = cv2.imwrite(candidate_path, crop)
                if ok:
                    crop_path = candidate_path
                else:
                    # если запись не удалась — не сохраняем путь в БД
                    crop_path = None
            except Exception:
                crop_path = None

        # Пишем в БД, игнорируя дубликаты
        cur.execute(
            """
            INSERT OR IGNORE INTO train_numbers
            (number, captured_at, video_path, frame_index, x_center, y_center, width, height, conf, crop_path)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """,
            (
                digits,
                datetime.utcnow().isoformat(timespec='seconds'),
                os.path.abspath(video_path_ocr),
                frame_index,
                cx, cy, ww, hh,
                conf,
                crop_path,
            )
        )
        conn.commit()

    # периодически чистим
    if frame_index % 50 == 0:
        gc.collect()

    frame_index += 1

cur.close()
conn.close()
print("Готово: поток обработан. Уникальные 8-значные номера внесены в БД:", db_path)



video 1/1 (frame 1/2176) d:\Documents\YOLOv12\IMG_6174.MP4: 640x640 (no detections), 95.6ms
video 1/1 (frame 2/2176) d:\Documents\YOLOv12\IMG_6174.MP4: 640x640 (no detections), 21.8ms
video 1/1 (frame 3/2176) d:\Documents\YOLOv12\IMG_6174.MP4: 640x640 (no detections), 25.8ms


C:\Users\user\AppData\Local\Temp\ipykernel_11972\1165013473.py:19: DeprecationWarning: The parameter `use_angle_cls` has been deprecated and will be removed in the future. Please use `use_textline_orientation` instead.
  paddle_ocr = PaddleOCR(det=True, rec=True, use_angle_cls=True, lang='en', show_log=False)


video 1/1 (frame 4/2176) d:\Documents\YOLOv12\IMG_6174.MP4: 640x640 (no detections), 22.4ms
video 1/1 (frame 5/2176) d:\Documents\YOLOv12\IMG_6174.MP4: 640x640 (no detections), 22.2ms
video 1/1 (frame 6/2176) d:\Documents\YOLOv12\IMG_6174.MP4: 640x640 (no detections), 22.5ms
video 1/1 (frame 7/2176) d:\Documents\YOLOv12\IMG_6174.MP4: 640x640 (no detections), 22.5ms
video 1/1 (frame 8/2176) d:\Documents\YOLOv12\IMG_6174.MP4: 640x640 (no detections), 22.4ms
video 1/1 (frame 9/2176) d:\Documents\YOLOv12\IMG_6174.MP4: 640x640 (no detections), 24.6ms
video 1/1 (frame 10/2176) d:\Documents\YOLOv12\IMG_6174.MP4: 640x640 (no detections), 22.4ms
video 1/1 (frame 11/2176) d:\Documents\YOLOv12\IMG_6174.MP4: 640x640 (no detections), 23.8ms
video 1/1 (frame 12/2176) d:\Documents\YOLOv12\IMG_6174.MP4: 640x640 (no detections), 22.3ms
video 1/1 (frame 13/2176) d:\Documents\YOLOv12\IMG_6174.MP4: 640x640 (no detections), 39.4ms
video 1/1 (frame 14/2176) d:\Documents\YOLOv12\IMG_6174.MP4: 640x640 (no det

In [ ]:



# 5.3) Экспорт БД (SQLite) в pandas DataFrame и сохранение в файлы
import os
import sqlite3
import pandas as pd

db_path = os.path.join("dataset", "inference", "train_numbers.db")

table_name = "train_numbers"
with sqlite3.connect(db_path) as conn:
    # Проверим, есть ли таблица
    exists = pd.read_sql_query(
        "SELECT name FROM sqlite_master WHERE type='table' AND name=?;",
        conn,
        params=(table_name,)
    )
    if exists.empty:
        raise RuntimeError(f"Таблица '{table_name}' не найдена в {db_path}")
    df_train_numbers = pd.read_sql_query(f"SELECT * FROM {table_name};", conn)

print(f"Загружено строк: {len(df_train_numbers)}")
df_train_numbers.head()

# Сохраним в CSV
csv_path = os.path.join("dataset", "inference", "train_numbers.csv")
df_train_numbers.to_csv(csv_path, index=False, encoding="utf-8")
print(f"CSV сохранён: {csv_path}")

# Сохраним в Parquet (если установлен pyarrow/fastparquet)
parquet_path = os.path.join("dataset", "inference", "train_numbers.parquet")
try:
    df_train_numbers.to_parquet(parquet_path, index=False)
    print(f"Parquet сохранён: {parquet_path}")
except Exception as e:
    print(f"Parquet не сохранён (нет зависимости?): {e}")

Загружено строк: 124
CSV сохранён: dataset\inference\train_numbers.csv
Parquet не сохранён (нет зависимости?): Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.


In [ ]:
# 5.2) Потоковый инференс + улучшенный OCR для 8-значных номеров
# - Мультивариантная предобработка (контраст/порог/морфология/масштаб)
# - Двойной OCR: PaddleOCR (приоритет) + EasyOCR (фоллбэк), учёт confidence
# - Выбор лучшего 8-значного подстрочного кандидата
# - Кластеризация похожих прочтений (Hamming<=2) и консенсус по позициям
# - Стабильные номера сохраняются ОДИН раз в CSV; кропы сохраняются всегда

import os
import csv
import gc
import cv2
import sys
import time
import math
import torch
import numpy as np
from datetime import datetime
from collections import Counter, defaultdict

# ------------------------- Параметры -------------------------
video_path_ocr = "IMG_6174.MP4"  # путь к видео
conf_thresh_det = 0.80           # фильтр детекции (повышен для снижения мусора)
iou_thresh_det = 0.5             # IoU для детектора
save_crops = True
crops_dir = os.path.join("dataset", "inference", "crops")
os.makedirs(crops_dir, exist_ok=True)

csv_path = os.path.join("dataset", "inference", "train_numbers.csv")
csv_header = [
    "number", "first_seen_at", "last_seen_at", "video_path",
    "first_frame", "last_frame", "x_center", "y_center",
    "width", "height", "mean_conf", "count", "sample_crop_path"
]

# Критерии стабилизации кластера
max_hamming_for_merge = 2  # насколько строки могут отличаться, чтобы считаться одной
min_votes_to_save = 3      # сколько подтверждений нужно, чтобы записать в CSV
min_conf_ocr = 0.30        # минимальный confidence OCR, используем прочтения ниже — редко

# Расширение ROI вокруг бокса (увелич. шанс целиком захватить номер)
roi_expand_w = 1.6
roi_expand_h = 1.3

# ------------------------- Инициализация OCR -------------------------
paddle_ocr = None
try:
    from paddleocr import PaddleOCR
    # Дет+реког, английская модель, без лишнего логирования
    paddle_ocr = PaddleOCR(det=True, rec=True, use_angle_cls=True, lang='en', show_log=False)
except Exception:
    paddle_ocr = None

reader = None
try:
    import easyocr
    reader = easyocr.Reader(['en'], gpu=torch.cuda.is_available())
except Exception:
    reader = None

DIGITS = set("0123456789")

# ------------------------- Вспомогательные функции -------------------------
def only_digits(text: str) -> str:
    return "".join(ch for ch in text if ch in DIGITS)

def best_8_digit_substring(text: str) -> str:
    """Возвращает лучшую (первую) 8-значную подстроку из digit-only текста."""
    digits = only_digits(text)
    if len(digits) < 8:
        return ""
    # простая эвристика: берём первое "окно" длины 8. Можно улучшить по частоте.
    for i in range(len(digits) - 7):
        candidate = digits[i:i+8]
        if len(candidate) == 8:
            return candidate
    return ""

def hamming_distance(a: str, b: str) -> int:
    if len(a) != len(b):
        return 8  # максимальная "плохая" дистанция
    return sum(1 for x, y in zip(a, b) if x != y)

# ------------------------- Предобработка изображения -------------------------
def to_3ch(image: np.ndarray) -> np.ndarray:
    if image is None:
        return image
    if len(image.shape) == 2:
        return cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    return image

def preprocess_variants(crop_bgr: np.ndarray) -> list:
    """Генерирует набор предобработанных вариантов ROI для OCR."""
    variants = []

    # Базовый грей, масштабирование
    gray = cv2.cvtColor(crop_bgr, cv2.COLOR_BGR2GRAY)
    for scale in (1.5, 2.0, 3.0):
        resized = cv2.resize(gray, None, fx=scale, fy=scale, interpolation=cv2.INTER_CUBIC)

        # CLAHE для повышения контраста
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        high_contrast = clahe.apply(resized)

        # Несколько бинаризаций
        thr_otsu = cv2.threshold(high_contrast, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
        thr_inv = cv2.threshold(high_contrast, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
        thr_adapt = cv2.adaptiveThreshold(high_contrast, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                          cv2.THRESH_BINARY, 31, 9)

        # Мягкое шумоподавление/усиление границ
        blur = cv2.GaussianBlur(high_contrast, (3, 3), 0)
        sharp = cv2.addWeighted(high_contrast, 1.5, blur, -0.5, 0)

        # Морфология для разрыва/слияния штрихов
        kernel = np.ones((3, 3), np.uint8)
        open_img = cv2.morphologyEx(thr_otsu, cv2.MORPH_OPEN, kernel, iterations=1)
        close_img = cv2.morphologyEx(thr_otsu, cv2.MORPH_CLOSE, kernel, iterations=1)

        # Собираем кандидатов
        candidates = [resized, high_contrast, thr_otsu, thr_inv, thr_adapt, sharp, open_img, close_img]
        for img in candidates:
            variants.append(to_3ch(img))

    # Убедиться, что оригинал (3ch) тоже пробуем
    variants.append(to_3ch(crop_bgr))
    return variants

# ------------------------- OCR над несколькими вариантами -------------------------
def ocr_with_paddle(image_bgr: np.ndarray) -> list:
    """Возвращает список (text, score) из PaddleOCR."""
    results = []
    if paddle_ocr is None:
        return results
    try:
        img_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
        ocr_res = paddle_ocr.ocr(img_rgb, cls=True)
        for line in (ocr_res or []):
            for det in (line or []):
                txt, score = det[1][0], det[1][1]
                if txt and score is not None:
                    results.append((txt, float(score)))
    except Exception:
        pass
    return results


def ocr_with_easyocr(image_bgr: np.ndarray) -> list:
    """Возвращает список (text, score) из EasyOCR."""
    results = []
    if reader is None:
        return results
    try:
        # detail=1 чтобы получить confidence; allowlist — только цифры
        res = reader.readtext(image_bgr, detail=1, paragraph=False, allowlist='0123456789')
        for _, txt, score in (res or []):
            if txt and score is not None:
                results.append((txt, float(score)))
    except Exception:
        pass
    return results


def ocr_digits_best8_from_variants(crop_bgr: np.ndarray) -> tuple[str, float]:
    """
    Прогоняет несколько предобработок через Paddle/Easy и выбирает лучшую 8-значную строку.
    Возвращает (digits8, confidence) или ("", 0.0)
    """
    best_digits = ""
    best_score = 0.0

    for variant in preprocess_variants(crop_bgr):
        # PaddleOCR сначала
        for txt, score in ocr_with_paddle(variant):
            cand = best_8_digit_substring(txt)
            if len(cand) == 8 and score >= best_score:
                best_digits, best_score = cand, float(score)
        # EasyOCR фоллбэк/дополнение
        for txt, score in ocr_with_easyocr(variant):
            cand = best_8_digit_substring(txt)
            if len(cand) == 8 and score >= best_score:
                best_digits, best_score = cand, float(score)

    return best_digits, float(best_score)

# ------------------------- Кластеризация прочтений -------------------------
class NumberCluster:
    def __init__(self, first_digits: str, frame_idx: int, cx: float, cy: float, w: float, h: float,
                 conf: float, crop_path: str | None):
        self.samples: list[str] = [first_digits]
        self.consensus: str = first_digits
        self.count: int = 1
        self.total_conf: float = conf
        self.first_seen_at: str = datetime.utcnow().isoformat(timespec='seconds')
        self.last_seen_at: str = self.first_seen_at
        self.first_frame: int = frame_idx
        self.last_frame: int = frame_idx
        self.cx: float = cx
        self.cy: float = cy
        self.w: float = w
        self.h: float = h
        self.sample_crop_path: str | None = crop_path
        self.saved_to_csv: bool = False

    def update(self, new_digits: str, frame_idx: int, cx: float, cy: float, w: float, h: float,
               conf: float, crop_path: str | None):
        self.samples.append(new_digits)
        self.count += 1
        self.total_conf += max(conf, 0.0)
        self.last_seen_at = datetime.utcnow().isoformat(timespec='seconds')
        self.last_frame = frame_idx
        # Обновим усреднённые координаты (сглаживаем)
        alpha = 0.25
        self.cx = (1 - alpha) * self.cx + alpha * cx
        self.cy = (1 - alpha) * self.cy + alpha * cy
        self.w  = (1 - alpha) * self.w  + alpha * w
        self.h  = (1 - alpha) * self.h  + alpha * h
        if crop_path and not self.sample_crop_path:
            self.sample_crop_path = crop_path
        self._recompute_consensus()

    def mean_conf(self) -> float:
        return self.total_conf / max(self.count, 1)

    def _recompute_consensus(self):
        # Мажоритарный выбор по каждой позиции (длина всегда 8)
        if not self.samples:
            return
        cols = list(zip(*self.samples))  # 8 колонок
        consensus_chars = []
        for col in cols:
            votes = Counter(col)
            consensus_chars.append(votes.most_common(1)[0][0])
        self.consensus = "".join(consensus_chars)


class NumberClusters:
    def __init__(self):
        self.clusters: list[NumberCluster] = []
        self.saved_numbers: set[str] = set()  # чтобы не писать дубликаты в CSV

    def find_best_cluster_idx(self, digits: str) -> int:
        best_i, best_dist = -1, 9
        for i, c in enumerate(self.clusters):
            d = hamming_distance(digits, c.consensus)
            if d < best_dist:
                best_i, best_dist = i, d
        return best_i if best_dist <= max_hamming_for_merge else -1

    def upsert(self, digits: str, frame_idx: int, cx: float, cy: float, w: float, h: float,
               conf: float, crop_path: str | None) -> NumberCluster:
        idx = self.find_best_cluster_idx(digits)
        if idx == -1:
            cl = NumberCluster(digits, frame_idx, cx, cy, w, h, conf, crop_path)
            self.clusters.append(cl)
            return cl
        cl = self.clusters[idx]
        cl.update(digits, frame_idx, cx, cy, w, h, conf, crop_path)
        return cl

    def mark_saved(self, number: str):
        self.saved_numbers.add(number)

    def already_saved(self, number: str) -> bool:
        return number in self.saved_numbers


# ------------------------- CSV утилиты -------------------------
def ensure_csv_header(path: str, header: list[str]):
    if not os.path.exists(path) or os.path.getsize(path) == 0:
        os.makedirs(os.path.dirname(path), exist_ok=True)
        with open(path, "w", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow(header)


def append_row_csv(path: str, row: list):
    with open(path, "a", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(row)


# ------------------------- Основной цикл: инференс + OCR -------------------------
# Требуется, чтобы переменная `model` была заранее инициализирована (YOLO v11/12/ultralytics)
if 'model' not in globals():
    raise RuntimeError("Ожидается, что YOLO `model` уже инициализирован в предыдущих ячейках.")

ensure_csv_header(csv_path, csv_header)
clusters = NumberClusters()

frame_index = 0

for r in model(source=video_path_ocr, stream=True, conf=conf_thresh_det, iou=iou_thresh_det, device=0):
    frame = getattr(r, 'orig_img', None)
    if frame is None:
        frame_index += 1
        continue
    h, w = frame.shape[:2]

    boxes = getattr(r, 'boxes', None)
    if boxes is None or len(boxes) == 0:
        frame_index += 1
        if frame_index % 50 == 0:
            gc.collect()
        continue

    xyxy = boxes.xyxy.cpu().numpy()
    confs = boxes.conf.cpu().numpy().reshape(-1)
    xywhn = boxes.xywhn.cpu().numpy()

    for i, (x1, y1, x2, y2) in enumerate(xyxy):
        conf_det = float(confs[i])
        if conf_det < conf_thresh_det:
            continue

        # Расширяем ROI по обеим осям и клипуем в пределах кадра
        box_w = (x2 - x1)
        box_h = (y2 - y1)
        cx_pix = (x1 + x2) / 2.0
        cy_pix = (y1 + y2) / 2.0
        exp_w = box_w * roi_expand_w
        exp_h = box_h * roi_expand_h
        nx1 = int(max(0, cx_pix - exp_w / 2.0))
        nx2 = int(min(w, cx_pix + exp_w / 2.0))
        ny1 = int(max(0, cy_pix - exp_h / 2.0))
        ny2 = int(min(h, cy_pix + exp_h / 2.0))
        if nx2 <= nx1 or ny2 <= ny1:
            continue

        crop = frame[ny1:ny2, nx1:nx2]
        if crop.size == 0:
            continue
        crop = crop.copy()  # гарантировать непрерывность памяти

        # OCR через мульти-предобработку и двойной движок
        digits8, conf_ocr = ocr_digits_best8_from_variants(crop)
        if len(digits8) != 8:
            continue
        if conf_ocr < min_conf_ocr:
            # Будем мягкими: допускаем, но такие чтения будут "перебиты" консенсусом
            pass

        # Нормализованные координаты для записи/кластеризации
        cx_n, cy_n, w_n, h_n = map(float, xywhn[i])

        # Сохраняем кроп кадра (в любом случае, чтобы потом можно было проверить)
        crop_path = None
        if save_crops:
            try:
                crop_name = f"frame_{frame_index:06d}_i{i}_{digits8}.jpg"
                candidate_path = os.path.join(crops_dir, crop_name)
                if cv2.imwrite(candidate_path, crop):
                    crop_path = candidate_path
            except Exception:
                crop_path = None

        # Обновляем/создаём кластер для текущего чтения
        cl = clusters.upsert(
            digits=digits8,
            frame_idx=frame_index,
            cx=cx_n, cy=cy_n, w=w_n, h=h_n,
            conf=conf_ocr, crop_path=crop_path
        )

        # Если кластер стабилен и ещё не сохранён — пишем в CSV (один раз)
        if (not cl.saved_to_csv) and (cl.count >= min_votes_to_save) and (not clusters.already_saved(cl.consensus)):
            row = [
                cl.consensus,
                cl.first_seen_at,
                cl.last_seen_at,
                os.path.abspath(video_path_ocr),
                cl.first_frame,
                cl.last_frame,
                round(cl.cx, 6),
                round(cl.cy, 6),
                round(cl.w, 6),
                round(cl.h, 6),
                round(cl.mean_conf(), 4),
                cl.count,
                cl.sample_crop_path or "",
            ]
            append_row_csv(csv_path, row)
            cl.saved_to_csv = True
            clusters.mark_saved(cl.consensus)

    if frame_index % 50 == 0:
        gc.collect()
    frame_index += 1

print("Готово: поток обработан. Стабилизированные 8-значные номера внесены в CSV:", csv_path)

